In [ ]:

import numpy as np
import random
from tqdm import tqdm
from collections import defaultdict, Counter
import sentencepiece as spm
import Levenshtein
import matplotlib.pyplot as plt
from Bio.Seq import Seq
import os
from Bio import SeqIO
import pandas as pd
import networkx as nx
from Levenshtein import distance as levenshtein_distance
import seaborn as sns
from joblib import Parallel, delayed
from Bio.Align import PairwiseAligner
import itertools
import nx_parallel as nxp
import math

Notes for this notebook:

- it is too big to run all the orgs at once. you have to run each org separately. Bigger genomes that aren't subsampled take a long time, even with parallelization.
-to run this, first run the sp_and_kmer_tokenizers.ipynb notebook (https://github.com/mccsandora/biotokens/blob/main/sp_and_kmer_tokenizers.ipynb) and find the optimal vocab size for each genome. then input that result into the first code cell.
- make sure that you run all code cells with random subgraphs multiple times to get a representative understanding of the network structures and their variations
- Ideally I think it would be interesting to try compression methods (similar to what they did in this paper: https://arxiv.org/pdf/2401.14025) after analysis to see if compression is different across dif domains, and if/how any big differences in the network structures might affect compressibility. It would be so cool to train some ML models to predict compressibility based on analysis results and network metrics, and better understand factors that influence compression efficiency (even though I think this analysis infers a lot of that), but idk if we'll have time for that
- Could be cool to investigate further into a method to map high degree tokens, high entropy regions and highly connected subgraphs to functional elements(I think maybe introns and exons would be easiest? Idk if that's doable but if it is can try to write code for this but I'll need a bio person's help.)
-still working on the entropy part

In [ ]:
import os

tokenizers_dir = "/Users/tiananoll-walker/Documents/biotokens/tokenizers"

def load_vocab(org, vocab_size, method):
    vocab_path = os.path.join(tokenizers_dir, f"{org}_{vocab_size}_{method}.vocab")
    with open(vocab_path, 'r') as f:
        vocab = [line.split('\t')[0] for line in f.readlines()[3:]] 
    return vocab

#org = "Theileria orientalis strain Shintoku"
#org =  "Nanobdella aerobiophila"
org =  "Candidatus Karelsulcia muelleri"
#org =  "Malassezia restricta"
#org =  "Caenorhabditis elegans"
#org = "Ostreococcus lucimarinus CCE9901"
#org =   "Rice yellow mottle virus satellite"
vocab_size = 6000
method =  'kmer' #or "sp"  

tokens = load_vocab(org, vocab_size, method)

In [ ]:
"""the levenshtein distance is a metric for measuring the difference between 2 sequences. it counts the minimum number of single character edits required to change one word into the other.
in our analysis, we use the lev distance to determine how closely related different tokens are. by constructing a graph where nodes are tokens and edges exist between
nodes that are levenshtein distance 1 apart, we can visualize and analyze the relationship between tokens"""

def is_distance_one(seq1, seq2):
    if abs(len(seq1) - len(seq2)) > 1:
        return False
    overlap1 = {seq1[:i] + seq1[i+1:] for i in range(len(seq1))}
    overlap2 = {seq2[:i] + seq2[i+1:] for i in range(len(seq2))}
    return not overlap1.isdisjoint(overlap2)

def process_token_pairs(tokens, idx):
    token1 = tokens[idx]
    edges = []
    for j in range(idx + 1, len(tokens)):
        token2 = tokens[j]
        if is_distance_one(token1, token2):
            edges.append((token1, token2))
    return edges

def build_lev_graph(tokens):
    G = nx.Graph()
    G.add_nodes_from(tokens)
    
    results = Parallel(n_jobs=-2)(delayed(process_token_pairs)(tokens, i) for i in range(len(tokens)))

    for edge_list in results:
        G.add_edges_from(edge_list)
    return G

#tokens=load_vocab("Nanobdella aerobiophila", 6000, method) 
#tokens = load_vocab("e_coli_genome", 37000)
#tokens=load_vocab("Caenorhabditis_elegans", 31000)
tokens = load_vocab("Candidatus Karelsulcia muelleri", 6000, method)
levenshtein_graph = build_lev_graph(tokens)
print(f"levenshtein graph has {len(levenshtein_graph.nodes())} nodes and {len(levenshtein_graph.edges())} edges")


In [ ]:
# 16 minutes for ecoli. not necessary because its so large but interesting to look at 
#visualizing subgraph that only has nodes w at least 5 edges connected to it 
def visualize_graph(G, threshold=5):
    filtered_nodes = [node for node, degree in G.degree() if degree >= threshold]
    if not filtered_nodes:
        print("no nodes meet threshold ")
        return
    
    G_filtered = G.subgraph(filtered_nodes)
    plt.figure(figsize=(12, 12))
    pos = nx.spring_layout(G_filtered, seed=42)
    nx.draw(G_filtered, pos, with_labels=True, node_size=100, font_size=8, node_color='pink', font_color='black', edge_color='gray')
    plt.title("levenshtein graph")
    plt.show()

visualize_graph(levenshtein_graph, threshold=5)

In [ ]:
"""degree distribution shows how many connections (edges) each node has to other nodes"""
#degree : num of edges connected to a node
#frequency : number of nodes w that degree

def plot_degree_distribution(G):
    degrees = [G.degree(n) for n in G.nodes()]
    plt.figure()
    plt.hist(degrees, bins=range(1, max(degrees)+1))
    plt.title("degree distribution")
    plt.xlabel("degree")
    plt.ylabel("frequency")
    plt.show()

plot_degree_distribution(levenshtein_graph)

avg_degree = np.mean([deg for node, deg in levenshtein_graph.degree()])
print(f"avg degree: {avg_degree}")

In [ ]:
#measures the similarity of connections in the graph w respect to the degree
#between -1 and 1, high val means that nodes w high degrees more likely to connect w other high degree nodes

ac = nx.degree_assortativity_coefficient(levenshtein_graph)
print(f"degree assortativity coefficient : {ac}")

In [ ]:


#visualizing nodes that have a degree >= to the avg degree +1 SD, highlighting the more connected nodes in the grap

avg_degree = np.mean([degree for _, degree in levenshtein_graph.degree()])
std_dev_degree = np.std([degree for _, degree in levenshtein_graph.degree()])

degree_threshold = avg_degree + std_dev_degree

def visualize_high_degree_nodes(G, degree_threshold):
    high_degree_nodes = [node for node, degree in G.degree() if degree >= degree_threshold]
    if not high_degree_nodes:
        print("no nodes meet threshold ")
        return
    
    G_high = G.subgraph(high_degree_nodes)
    plt.figure(figsize=(12, 12))
    pos = nx.spring_layout(G_high, seed=42)
    nx.draw(G_high, pos, with_labels=True, node_size=100, font_size=8, node_color='pink', font_color='black', edge_color='gray')
    plt.title("high degree nodes graph")
    plt.show()

high_d_nodes = visualize_high_degree_nodes(levenshtein_graph, degree_threshold)


In [ ]:
"""centrality measures help identify the most important nodes in a network. We can use these measures to analyze the role/importance of each token in the fitness landscape."""
#12 mins to run for nano
#120 mins for ecoli

def compute_centrality_measures(G, max_iter=1000, tol=1e-06):
    centrality_measures = {
        "degree_centrality": nx.degree_centrality(G),
        "closeness_centrality": nx.closeness_centrality(G),
        "betweenness_centrality": nxp.betweenness_centrality(G),
    }
    
    try:
        centrality_measures["eigenvector_centrality"] = nx.eigenvector_centrality(G, max_iter=max_iter, tol=tol)
    except nx.PowerIterationFailedConvergence as e:
        print(f"eig calculation failed to converge: {e}")
    
    return centrality_measures

centrality_measures = compute_centrality_measures(levenshtein_graph)
#for measure, values in centrality_measures.items():
#    print(f"{measure}:")
#    for node, value in values.items():
#        print(f"{node}: {value:.4f}")
#    print()

def plot_centrality_distribution(centrality_dict, title):
    values = list(centrality_dict.values())
    plt.figure(figsize=(10, 6))
    plt.hist(values, bins=30, color='purple', edgecolor='black')
    plt.title(title)
    plt.xlabel('centrality val')
    plt.ylabel('frequency')
    plt.show()

plot_centrality_distribution(centrality_measures["degree_centrality"], "degree centrality dist")
plot_centrality_distribution(centrality_measures["closeness_centrality"], "closeness centrality dist")
plot_centrality_distribution(centrality_measures["betweenness_centrality"], "betweenness centrality dist")
plot_centrality_distribution(centrality_measures["eigenvector_centrality"], "eigenvector centrality dist")

In [ ]:
#11 mins for ecoli

def detect_communities(G):
    communities = list(nx.algorithms.community.greedy_modularity_communities(G))
    return communities

communities = detect_communities(levenshtein_graph)
print(f"found {len(communities)} communities ")
#for i, community in enumerate(communities):
#    print(f"community {i + 1}: {sorted(community)}")


community_sizes = [len(c) for c in communities]
plt.figure(figsize=(10, 6))
plt.hist(community_sizes, bins=50, color='blue', edgecolor='black')
plt.title("community size distribution")
plt.xlabel("community size")
plt.ylabel("frequency")
plt.show()

print(f"mean com size: {np.mean(community_sizes)}")
print(f"median com size: {np.median(community_sizes)}")
print(f"standard dev of com sizes: {np.std(community_sizes)}")
print(f"biggest com size: {np.max(community_sizes)}")
print(f"smallest com size: {np.min(community_sizes)}")

In [ ]:
"""path lengths measure the shortest num of edges to travel between nodes in the graph. so distance of 1 is a direct neighbor, 2 is two doors down, more than 2 means there are multiple intermediary tokens required to connect them  """
#40 mins for ecoli

#diameter: longest shortest path between any two nodes
#avg path length: any two nodes in the graph are about ~avg path length~ apart

def compute_path_lengths(G):
    path_lengths = dict(nxp.all_pairs_shortest_path_length(G))
    return path_lengths

path_lengths = compute_path_lengths(levenshtein_graph)
print(f"computed path lengths for {len(path_lengths)} nodes ")
#for node, lengths in path_lengths.items():
#    print(f"{node}: {lengths}")


all_path_lengths = [length for lengths in path_lengths.values() for length in lengths.values()]

plt.figure(figsize=(10, 6))
sns.histplot(all_path_lengths, bins=50, kde=False, color='blue')
plt.title('path length distribution')
plt.xlabel('path length')
plt.ylabel('frequency')
plt.show()

if nx.is_connected(levenshtein_graph):
    avg_p_len = nx.average_shortest_path_length(levenshtein_graph)
    diameter = nx.diameter(levenshtein_graph)
else:
    largest_cc = max(nx.connected_components(levenshtein_graph), key=len)
    subgraph = levenshtein_graph.subgraph(largest_cc)
    avg_p_len = nx.average_shortest_path_length(subgraph)
    diameter = nx.diameter(subgraph)

print(f"avg path length: {avg_p_len}")
print(f"diameter: {diameter}")

In [ ]:
"""the clustering coefficient measures the degree to which nodes in a graph tend to cluster together.
It's calculated for each node and represents the ratio of the num of actual edges between the node’s 
neighbors to the number of possible edges between the node’s neighbors. A high clustering coefficient 
indicates that a node’s neighbors are also closely connected to each other."""



def compute_clustering_coefficient(G):
    clustering_coefficient = nx.clustering(G)
    return clustering_coefficient

clustering_coefficient = compute_clustering_coefficient(levenshtein_graph)
#print("clustering coefficients:")
#for node, coefficient in clustering_coefficient.items():
#    print(f"{node}: {coefficient:.4f}")

def plot_clustering_coefficient_distribution(clustering_coefficient):
    coefficients = list(clustering_coefficient.values())
    plt.figure()
    plt.hist(coefficients, bins=20, color='pink', edgecolor='black')
    plt.title("clustering coefficient distribution")
    plt.xlabel("clustering coefficient")
    plt.ylabel("frequency")
    plt.show()

plot_clustering_coefficient_distribution(clustering_coefficient)

avg_cc = nx.average_clustering(levenshtein_graph)
print(f"avg cc: {avg_cc}")

In [ ]:
"""ratio of num of edges to num of possible edges. measures how interconnected the graph is"""

density = nx.density(levenshtein_graph)
print(f"graph density: {density}")

In [ ]:
#random sample

def random_sub(G, size):
    nodes = random.sample(list(G.nodes()), size)
    subgraph = G.subgraph(nodes)
    return subgraph

random_subgraph = random_sub(levenshtein_graph, 100)

plt.figure(figsize=(12, 12))
pos = nx.spring_layout(random_subgraph)
nx.draw(random_subgraph, pos, with_labels=True, node_size=500,
        font_size=10, node_color='pink', font_color='black', edge_color='gray')
plt.title('random subgraph')
plt.show()

In [ ]:
# random sample via breadth first search. it's random but biased toward connectivity

def connected_random_subgraph(G, size):
    start_node = random.choice(list(G.nodes()))
    visited = set([start_node])
    queue = [start_node]
    
    while queue and len(visited) < size:
        node = queue.pop(0)
        neighbors = list(G.neighbors(node))
        random.shuffle(neighbors)
        for neighbor in neighbors:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)
                if len(visited) == size:
                    break
    
    return G.subgraph(visited)

random_subgraph = connected_random_subgraph(levenshtein_graph, 100)

plt.figure(figsize=(12, 12))
pos = nx.spring_layout(random_subgraph)
nx.draw(random_subgraph, pos, with_labels=True, node_size=500,
        font_size=8, node_color='pink', font_color='black', edge_color='gray')
plt.title('random connected subgraph via bfs')
plt.show()

In [ ]:
# ego graph centered around a specific randomly selected node w radius of 2(the ego node, all nodes directly connected to it, and all nodes connected to those nodes)
#run a few times

def ego_sub(G, node, radius):
    subgraph = nx.ego_graph(G, node, radius=radius)
    return subgraph

ego_node = random.choice(list(levenshtein_graph.nodes))
ego_subgraph = ego_sub(levenshtein_graph, ego_node, radius=2)


plt.figure(figsize=(12, 12))
pos = nx.spring_layout(ego_subgraph)
nx.draw(ego_subgraph, pos, with_labels=True, node_size=100, font_size=8, node_color='lightgreen', font_color='black', edge_color='gray')
plt.title(f"ego graph centered around token {ego_node}")
plt.show()

In [ ]:
"""the smith-waterman algo is used for local sequence alignment. it finds the optimal local alignment between two sequences by identifying regions of high similarity.
unlike the needleman-wunsch algo, which performs a global alignment, the smith-waterman algorithm focuses on the most similar parts of the sequences, allowing for gaps.
essentially we measure the local similarity between different tokens by computing alignment scores. red regions indicate higher local sequence similarity between those token pairs"""

subsampled_tokens = random.sample(tokens, 100)

def calculate_sw_score(token1, token2):
    aligner = PairwiseAligner()
    aligner.mode = 'local'
    aligner.match_score = 2
    aligner.mismatch_score = -1
    aligner.open_gap_score = -2
    aligner.extend_gap_score = -1
    alignments = aligner.align(token1, token2)
    return (token1, token2, alignments.score)

def process_pairs(tokens):
    pairs = [(tokens[i], tokens[j]) for i in range(len(tokens)) for j in range(i + 1, len(tokens))]
    
    results = Parallel(n_jobs=-2)(delayed(calculate_sw_score)(pair[0], pair[1]) for pair in tqdm(pairs, total=len(pairs)))
    return results

results = process_pairs(subsampled_tokens)

df = pd.DataFrame(results, columns=["token1", "token2", "score"])

pivot_table = df.pivot_table(index="token1", columns="token2", values="score", fill_value=0)
plt.figure(figsize=(16, 14))
sns.heatmap(pivot_table, annot=False, cmap="coolwarm", cbar=True, linewidths=.5, linecolor='gray')
plt.title("smith-waterman score heatmap")
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.show()

In [ ]:
"""the needleman wunsch algo is another algo for sequence alignment. its used to align protein or nucleotide sequences by finding the optimal match between them, allowing for gaps. 
the algo works by creating a score matrix and finding the highest scoring alignment. in our fitness landscape analysis we use
the NW algo to measure the similarity between dif tokens by computing alignment scores. """

subsampled_tokens = random.sample(tokens, 100)

def needleman_wunsch(seq1, seq2, match=1, mismatch=-1, gap=-1):
    n = len(seq1)
    m = len(seq2)
    score = np.zeros((n + 1, m + 1))
    
    for i in range(1, n + 1):
        score[i][0] = score[i - 1][0] + gap
    for j in range(1, m + 1):
        score[0][j] = score[0][j] + gap

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            match_mismatch = match if seq1[i - 1] == seq2[j - 1] else mismatch
            score[i][j] = max(
                score[i - 1][j - 1] + match_mismatch,
                score[i - 1][j] + gap,
                score[i][j - 1] + gap
            )
    return score[n][m]

def compute_nw_score(pair):
    seq1, seq2 = pair
    return (seq1, seq2, needleman_wunsch(seq1, seq2))

def process_pairs(tokens):
    pairs = [(tokens[i], tokens[j]) for i in range(len(tokens)) for j in range(i + 1, len(tokens))]
    results = Parallel(n_jobs=-2)(delayed(compute_nw_score)(pair) for pair in tqdm(pairs))
    return results

nw_scores = process_pairs(subsampled_tokens)

#for seq1, seq2, score in nw_scores:
#    print(f"NW score between {seq1} and {seq2}: {score}")

df = pd.DataFrame(nw_scores, columns=["token1", "token2", "score"])
pivot_table = df.pivot_table(index="token1", columns="token2", values="score", fill_value=0)

plt.figure(figsize=(16, 14))
sns.heatmap(pivot_table, annot=False, cmap="coolwarm", cbar=True, linewidths=.5, linecolor='gray')
plt.title("needleman-wunsch score heatmap")
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.show()

In [ ]:

def shannon_entropy(tokens):
    token_counts = Counter(tokens)
    total_count = len(tokens)
    entropy = -sum((count / total_count) * math.log2(count / total_count) for count in token_counts.values())
    return entropy

genome_entropy = shannon_entropy(tokens)
print(f"shannon entropy for entire genome: {genome_entropy}")

In [ ]:
#I don't think this code is right lol, so I'm commenting it out until further notice. 

window_size = 1000
step_size = 500
entropies = []
positions = []

for i in range(0, len(tokens) - window_size + 1, step_size):
    window_tokens = tokens[i:i + window_size]
    window_entropy = shannon_entropy(window_tokens)
    entropies.append(window_entropy)
    positions.append(i)

high_entropy_threshold = np.percentile(entropies, 95)
print(f"high entropy threshold (95th percentile): {high_entropy_threshold}")

high_entropy_regions = [(positions[i], positions[i] + window_size) for i, entropy in enumerate(entropies) if entropy >= high_entropy_threshold]
print(f"num of high entropy regions: {len(high_entropy_regions)}")
print(f"high entropy regions: {high_entropy_regions}")

def merge_regions(regions):
    if not regions:
        return []

    regions = sorted(regions, key=lambda x: x[0])
    merged_regions = [regions[0]]
    
    for current in regions:
        last = merged_regions[-1]
        if current[0] <= last[1]:
            merged_regions[-1] = (last[0], max(last[1], current[1]))
        else:
            merged_regions.append(current)
    
    return merged_regions

merged_high_entropy_regions = merge_regions(high_entropy_regions)
#print(f"merged high entropy regions: {merged_high_entropy_regions}")

plt.figure(figsize=(10, 6))
plt.plot(positions, entropies, marker='o', linestyle='-', label='Entropy')

for start, end in merged_high_entropy_regions:
    plt.axvspan(start, end, color='red', alpha=0.3, label='High Entropy Region' if start == merged_high_entropy_regions[0][0] else "")

plt.title("shannon entropy across the tokens")
plt.xlabel("position in token list")
plt.ylabel("shannon entropy")
plt.legend()
plt.show()